라이브러리 불러오기

In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By

#명시적 대기
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

다음 카카오맵 불러오기 > 검색어 입력 > 검색버튼 클릭

피플카존 저장

In [34]:
#다음 카카오맵 페이지 불러오기
main_url = 'https://map.kakao.com/'

In [35]:
keyword = input('검색어 입력:')

검색어 입력:피플존


In [36]:
driver = webdriver.Chrome(executable_path='C:/★팀프로젝트/크롤링/chromedriver.exe')

In [37]:
driver.get(main_url)

In [38]:
#검색창에 쏘카 입력하기
driver.find_element_by_id('search.keyword.query').send_keys(keyword)

In [39]:
#검색버튼 누르기
driver.find_element_by_xpath('/html/body/div[2]/div/div/form/fieldset/div[1]/button').click()
#5초 대기
time.sleep(5)

In [40]:
totalCount = driver.find_element_by_css_selector('#info\.search\.place\.cnt')
print('검색 갯수',totalCount.text)
#검색개수 ,삭제하고  정수로 변환
totalCount = int(totalCount.text.replace(',',''))
print(totalCount)

검색 갯수 388
388


리스트 데이터 저장

한 페이지당 selector copy를 하면 페이지당 1에서 16개임 (4번은 광고들어가서 빼면 15개 ㅇㅇ)

페이지 출력

In [41]:
#총 페이지수 
totalPage = round(totalCount/15,0)
totalPage

26.0

In [42]:
#다음 누를 횟수
import math
math.ceil(totalPage/5)

6

In [43]:
#마지막 페이지에 남는 개수
totalCount%15

13

In [44]:
df_empty = pd.DataFrame({'거점명' : [],'주소':[]})
df_empty

,거점명,주소


In [45]:
all_empty = pd.DataFrame({'거점명' : [],'주소':[]})
all_empty

,거점명,주소


In [46]:
#1페이지에서 장소 더보기 클릭
driver.find_element_by_css_selector('#info\.search\.place\.more').click()
time.sleep(2)

#다음 클릭
for next_click in range (1,(math.ceil(totalPage/5))):
    
    for i in range(1,6):
        driver.find_element_by_css_selector("#info\.search\.page\.no"+str(i)).click()
        time.sleep(2)
        
        garage_list = []
        address_list = []
        
        for j in range(1,17):
            if j != 4:
                #차고지명 
                garage = driver.find_element_by_css_selector('#info\.search\.place\.list > li:nth-child('+ str(j) + ') > div.head_item.clickArea > strong > a.link_name').text
                garage_list.append(garage)
                
                #주소
                address = driver.find_element_by_css_selector('#info\.search\.place\.list > li:nth-child(' +str(j) +') > div.info_item > div.addr > p:nth-child(1)').text
                address_list.append(address)
                #print(garage_list)
                #print(address_list)
                #list 짝짓기 
                
        data_tuples = list(zip(garage_list,address_list))
        #데이터프레임으로 만들기
        df = pd.DataFrame(data_tuples, columns=['거점명','주소'])
        
        #데이터프레임 합치기
        if i == 1:
            df_final = df_empty.append(df, ignore_index=True)
        else:
            df_final = df_final.append(df, ignore_index=True)

            #print(df_final)
    if next_click == 1:
        all_df_final = all_empty.append(df_final,ignore_index=True)
    else:
        all_df_final = all_df_final.append(df_final,ignore_index=True)
    #다음 클릭하기
    driver.find_element_by_css_selector('#info\.search\.page\.next').click()

driver.find_element_by_css_selector('#info\.search\.page\.no1').click()


##############마지막 페이지 #########################
garage_list = []
address_list= []
for last in range(1,totalCount%15+2 ):
    
    if last != 4:
        #차고지명 
        garage = driver.find_element_by_css_selector('#info\.search\.place\.list > li:nth-child('+ str(last) + ') > div.head_item.clickArea > strong > a.link_name').text
        garage_list.append(garage)
        #주소
        address = driver.find_element_by_css_selector('#info\.search\.place\.list > li:nth-child(' +str(last) +') > div.info_item > div.addr > p:nth-child(1)').text
        address_list.append(address)
        #print(garage_list)
        #print(address_list)
        #list 짝짓기 
                
data_tuples = list(zip(garage_list,address_list))
        #데이터프레임으로 만들기
df = pd.DataFrame(data_tuples, columns=['거점명','주소'])
        
all_df_final = all_df_final.append(df,ignore_index=True)

        
    #info\.search\.place\.list > li:nth-child(14) > div.head_item.clickArea > strong > a.link_name
print(all_df_final)

                             거점명                    주소
0             피플존 디지털미디어시티역 5번출구      서울 은평구 증산동 222-6
1                 피플존 망원동노외공영주차장       서울 마포구 월드컵로 131
2              피플존 아만티호텔 서울 지하3층       서울 마포구 월드컵북로 31
3               피플존 창천동삼거리(화이트빌)    서울 서대문구 창천동 100-26
4             피플존 한국항공대(방문자 주차장)   경기 고양시 덕양구 항공대학로 76
..                           ...                   ...
383             피플존 동아대정문앞교직원주차장   부산 사하구 낙동대로550번길 15
384             피플존 여수엑스포역 공영주차장       전남 여수시 덕충동 61-7
385       피플존 인제대학교 늘빛관 옆 주차장 3층  경남 김해시 어방동 607 인제대학교
386        피플존 부경대대연캠퍼스옆(에이스고시원)      부산 남구 용소로21번길 13
387  피플존 전남대 여수캠퍼스 체육관 운동장 뒤 주차장     전남 여수시 둔덕동 산 96-1

[388 rows x 2 columns]


In [47]:
all_df_final

,거점명,주소
0,피플존 디지털미디어시티역 5번출구,서울 은평구 증산동 222-6
1,피플존 망원동노외공영주차장,서울 마포구 월드컵로 131
2,피플존 아만티호텔 서울 지하3층,서울 마포구 월드컵북로 31
3,피플존 창천동삼거리(화이트빌),서울 서대문구 창천동 100-26
4,피플존 한국항공대(방문자 주차장),경기 고양시 덕양구 항공대학로 76
...,...,...
383,피플존 동아대정문앞교직원주차장,부산 사하구 낙동대로550번길 15
384,피플존 여수엑스포역 공영주차장,전남 여수시 덕충동 61-7
385,피플존 인제대학교 늘빛관 옆 주차장 3층,경남 김해시 어방동 607 인제대학교
386,피플존 부경대대연캠퍼스옆(에이스고시원),부산 남구 용소로21번길 13


In [49]:
all_df_final.to_csv('피플존_전국_카카오맵_크롤링.csv',encoding='cp949')